<img src="https://raw.githubusercontent.com/paular143/images_notebooks/master/logo-sin-nombre.png" alt="Universidad de Los Andes" style="width: 90px;" align="left" frameborder="300"/>

**UNIVERSIDAD DE LOS ANDES** <br>
**FACULTAD DE ECONOMÍA - EDUCACIÓN CONTINUADA** <br>
**Análisis espacial de datos y sus aplicaciones en Python 2021** <br>
**PROFESORES:** Germán González <br><br>


# Clase 6: Dbscan


In [1]:
# !pip3 install shapely
# !pip3 install pyproj
# !pip3 install folium
# !pip3 install pandas
# !pip3 install requests
# !pip3 install geopy
# !pip3 install geojson
# !pip3 install xlrd
# !pip3 install sklearn

import pyproj
from functools import partial
import shapely.geometry
from shapely.ops import transform
from shapely.geometry.polygon import Polygon 
from shapely.geometry import Point, MultiPoint 
from geopy.distance import great_circle
import folium
import geojson
import json
import pandas as pd
import re
import geopy
import requests
import time
import numpy as np
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN



/Users/germangonzalez/.pyenv/versions/3.7.5/envs/motus/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Comercios de bogotá: Camara de Comercio 2016 - Submuestra 250 comercios - Usaquen

In [2]:
Lugares_usaquen_sample = pd.read_pickle('Lugares_sample.gzip',compression='gzip')

In [3]:
Lugares_usaquen_sample

,ID,Nombre,Actividad,Direccion,Latitude,Longitude,Point,Grupo
0,0,TU TIENDA . U,Alojamiento y servicios de comida,CL 131 No 58 35 IN 3 AP 310,4.777510,-74.041255,POINT (-74.04125490482421 4.777510416416243),0
1,1,SERVICIOS INTEGRALES DE LOS ANDES,Actividades de atencionde la salud humana y de...,CLL 184 NO. 41-61 INT 1 APTO 402,4.760924,-74.043020,POINT (-74.04302011142549 4.760923651396257),1
2,2,VELO'S BICYCLE,Construccion,CL 164 A . 8 C - 47,4.740671,-74.027962,POINT (-74.02796248415774 4.740670707074391),2
3,3,AGROPECUARIA PIGS CATTLE TROPICAL,"Agricultura, ganader¡a, caza, silvicultura y p...",CR 55 No 151 69,4.720658,-74.039057,POINT (-74.03905669588293 4.720657536006083),3
4,4,PUNTO DE VENTA CALLE 150,Industrias manufactureras,AV 19 N.150 - 04,4.731748,-74.044676,POINT (-74.04467582597005 4.731747614581696),3
...,...,...,...,...,...,...,...,...
245,245,CORPO BELLO,Actividades de atencionde la salud humana y de...,CL 127 NO. 19 A 44,4.706272,-74.050034,POINT (-74.05003421824934 4.706272005032837),4
246,246,DENTAL CLINIC COLOMBIA,Actividades de atencionde la salud humana y de...,CR 14 A . 112 96 CONS 402,4.694596,-74.039045,POINT (-74.03904480218857 4.694596429824847),4
247,247,RISTORANTE LA BRUSCHETA,Alojamiento y servicios de comida,DG 109 NO. 14-61,4.691747,-74.043599,POINT (-74.04359933434091 4.691746978251736),4
248,248,TO BLOSSOM,Otras actividades de servicios,CR 19 . 122 49 LC 6,4.702478,-74.049456,POINT (-74.0494559466498 4.702477665611049),4


## Visualización Mapa

In [4]:
mapa = folium.Map(location=[4.729494,-74.049345],zoom_start=15,tiles='cartodbpositron')

for j in range(0,len(Lugares_usaquen_sample)):
    punto = Lugares_usaquen_sample.iloc[j] 
    folium.Marker(
        [punto['Latitude'],punto['Longitude']]
    ).add_to(mapa)


In [5]:
mapa

In [8]:
####### ------ Transform coords ------- #########
coords_to_clusterize = np.array(list(pd.DataFrame(Lugares_usaquen_sample.apply(lambda x: [float(x['Latitude']),float(x['Longitude'])],axis=1))[0]))
coords_radians = np.radians(coords_to_clusterize)



In [76]:
# Distancia máxima (1.5 kilómetros en este ejemplo) que los 
# puntos pueden estar el uno del otro para ser considerados un grupo
Metros = 300
Kilometros_cluster = Metros/1000

#### ----- Radio del mundo ------ ######
epsilon = 6371.0088
####### ------- Se calcula el epsilon en radiales ------- ########
epsilon = Kilometros_cluster/Kilometros_radio_mundo

In [77]:
epsilon

4.708830413167849e-05

In [78]:
########## ---------- Calcular clusters  --------- ##########
Clusters = DBSCAN(eps=epsilon,algorithm='ball_tree',metric='haversine',n_jobs=-1, min_samples=5)
Clusters.fit(coords_radians) 

DBSCAN(algorithm='ball_tree', eps=4.708830413167849e-05, metric='haversine',
       n_jobs=-1)

In [79]:
len(coords_radians)

250

In [80]:
len(Clusters.labels_)

250

In [81]:
Clusters.labels_

array([-1,  0,  1,  2,  3, -1,  4,  5, -1,  5, 10,  1, -1, -1,  6,  0,  6,
        7,  6,  1,  5, -1,  5,  5,  1,  8,  7, -1,  6, -1, -1,  0,  4,  1,
       -1, -1, -1,  6,  1,  8, -1, -1, -1,  8,  0,  7, -1, -1, -1, -1,  0,
        8,  6,  9,  2, -1, 10, -1, -1,  6, -1, -1, -1,  4, -1,  1,  6,  1,
        8, -1, -1, 11,  2,  1,  1, 12, -1, -1, -1,  9, -1, -1,  6,  8,  1,
       -1, -1,  6,  1,  6, -1,  5, -1, 11, 13,  3, 13, -1, -1,  5, -1,  6,
       -1, -1, -1, -1,  5,  8, 10,  1,  3,  6,  4, -1, -1,  9, 11,  4,  5,
        0,  5, -1, -1, -1,  4, -1, -1, 12,  0,  6,  7, 12, -1, -1,  4,  1,
       -1,  0, -1, -1,  1,  1, -1,  6,  1,  6, -1, -1,  6, -1, -1,  9,  8,
        6, -1, -1, -1, -1,  1,  3,  5,  1,  1,  4, -1,  5, -1,  6,  3, -1,
       -1, -1,  1,  4,  8, -1, -1, -1,  0,  8, -1, 13,  8,  6, 10, 12, -1,
        4,  7, -1,  6,  1,  9,  3,  4, -1,  2, -1,  1, 11, -1,  6, -1, -1,
        3, -1,  6,  8,  6, -1, -1,  8, -1, -1,  6, -1, -1, -1,  8, -1,  5,
       -1, 10,  8,  1, -1

## Se remplazan el número de lugares dentro 

In [82]:
#### ------ Se extraen las etiquetas de cada lot, lat ----- ####
Etiquetas_clusters = Clusters.labels_
#### ------ Asignar la etiqueta a cada observación ----- ###### 
Lugares_usaquen_sample['Grupo'] = Etiquetas_clusters

##### -------- Contar el número de observaciones por cluster ------ ##### 
Indice_clusters = pd.DataFrame(pd.value_counts(pd.DataFrame(Etiquetas_clusters)[0]).sort_index())[0]

### ------- Numero de grupos creados ---- ##### 
num_clusters = len(set(Etiquetas_clusters))


In [83]:
Lugares_usaquen_sample

,ID,Nombre,Actividad,Direccion,Latitude,Longitude,Point,Grupo
0,0,TU TIENDA . U,Alojamiento y servicios de comida,CL 131 No 58 35 IN 3 AP 310,4.777510,-74.041255,POINT (-74.04125490482421 4.777510416416243),-1
1,1,SERVICIOS INTEGRALES DE LOS ANDES,Actividades de atencionde la salud humana y de...,CLL 184 NO. 41-61 INT 1 APTO 402,4.760924,-74.043020,POINT (-74.04302011142549 4.760923651396257),0
2,2,VELO'S BICYCLE,Construccion,CL 164 A . 8 C - 47,4.740671,-74.027962,POINT (-74.02796248415774 4.740670707074391),1
3,3,AGROPECUARIA PIGS CATTLE TROPICAL,"Agricultura, ganader¡a, caza, silvicultura y p...",CR 55 No 151 69,4.720658,-74.039057,POINT (-74.03905669588293 4.720657536006083),2
4,4,PUNTO DE VENTA CALLE 150,Industrias manufactureras,AV 19 N.150 - 04,4.731748,-74.044676,POINT (-74.04467582597005 4.731747614581696),3
...,...,...,...,...,...,...,...,...
245,245,CORPO BELLO,Actividades de atencionde la salud humana y de...,CL 127 NO. 19 A 44,4.706272,-74.050034,POINT (-74.05003421824934 4.706272005032837),13
246,246,DENTAL CLINIC COLOMBIA,Actividades de atencionde la salud humana y de...,CR 14 A . 112 96 CONS 402,4.694596,-74.039045,POINT (-74.03904480218857 4.694596429824847),-1
247,247,RISTORANTE LA BRUSCHETA,Alojamiento y servicios de comida,DG 109 NO. 14-61,4.691747,-74.043599,POINT (-74.04359933434091 4.691746978251736),6
248,248,TO BLOSSOM,Otras actividades de servicios,CR 19 . 122 49 LC 6,4.702478,-74.049456,POINT (-74.0494559466498 4.702477665611049),11


In [84]:
##### -------- Crear lista con las coordenadas ----- ###### 
clusters = pd.Series([coords_to_clusterize[Etiquetas_clusters == n] for n in list(set(Etiquetas_clusters))])

In [85]:
Centroide = MultiPoint(clusters[0]).centroid.coords[0]

## Centroides

In [86]:
####### ------ función para encontrar las coordenadas de los centroides ------- ###### 
def Obtener_centro_clusters(cluster):
    ### ------- Encontrar el centroide de la figura ------ ####### 
    Centroide = MultiPoint(cluster).centroid.coords[0]
    ###### -------- Calcular distancia a un punto real -------- ######
    ##### ----------- Encontrar distancia minima ----- ####### 
    centermost_point = min(cluster , key=lambda x: great_circle(x , Centroide).m)
    return tuple(centermost_point)




In [87]:
###### ------- Calculate centroid ------- #######
Coordenadas_centroides = list( map(lambda x: Obtener_centro_clusters(x),clusters))


In [88]:
Coordenadas_centroides

[(4.759196599756379, -74.04443770863939),
 (4.737997294960866, -74.02720056277816),
 (4.72113064380087, -74.04012817401448),
 (4.730624982573526, -74.04267864202306),
 (4.721131719017022, -74.04608122136744),
 (4.739383531227223, -74.04357168419646),
 (4.695062793133971, -74.04213181595006),
 (4.685866650782341, -74.04764047481679),
 (4.694661133263812, -74.03479880989529),
 (4.773824757701787, -74.04121290319479),
 (4.76013649035759, -74.02816496621942),
 (4.700703042674348, -74.05039663816082),
 (4.694277377410958, -74.04956588876247),
 (4.706272005032837, -74.05003421824934),
 (4.731869560654983, -74.03863562137329)]

In [89]:
### ----- Extraer las latitude y longitudes 
Latitudes = list(map(lambda x: x[0],Coordenadas_centroides))
Longitudes = list(map(lambda x: x[1],Coordenadas_centroides)) 
### --------- Consolidar dataframe ---- ### 
Centroides_info = pd.DataFrame({'Longitude':Longitudes, 'Latitude':Latitudes})
### ------- Buscar información del centroide ------- #### 
Centroides = Centroides_info.apply(lambda row: Lugares_usaquen_sample[(Lugares_usaquen_sample['Latitude']==row['Latitude']) & (Lugares_usaquen_sample['Longitude']==row['Longitude'])].iloc[0], axis=1)
Centroides = pd.DataFrame(Centroides)




In [90]:
Centroides

,ID,Nombre,Actividad,Direccion,Latitude,Longitude,Point,Grupo
0,128,COLITALIA TALLERES CALLE 183,Comercio al por mayor y al por menor; reparaci...,AUT NORTE NO. 183 A 58,4.759197,-74.044438,POINT (-74.04443770863939 4.759196599756379),0
1,236,ROPA PARA NINOS IN FASHION KIDS,Comercio al por mayor y al por menor; reparaci...,CL 162 NO. 8 - 17,4.737997,-74.027201,POINT (-74.02720056277816 4.737997294960866),1
2,54,MAIL BOXES ETC CEDRITOS,Transporte y almacenamiento,CL 140 NO. 13-22,4.721131,-74.040128,POINT (-74.04012817401448 4.72113064380087),2
3,110,COMERCIALIZADORA DE PAPA LA PASTUSA,Actividades de organizaciones y entidades extr...,TRV 27 B NO. 148-20,4.730625,-74.042679,POINT (-74.04267864202306 4.730624982573526),3
4,6,TIENDA LENA,Comercio al por mayor y al por menor; reparaci...,AK 19 NO. 136A-06 LC 233 34,4.721132,-74.046081,POINT (-74.04608122136744 4.721131719017022),4
5,7,RAZA AUDIOVISUAL,Informacion y comunicaciones,CR 19 A NO. 159-70 IN 4-70 301,4.739384,-74.043572,POINT (-74.04357168419646 4.739383531227223),5
6,190,MICUIDADO.COM,Actividades de organizaciones y entidades extr...,CR 14 A NO. 112-45,4.695063,-74.042132,POINT (-74.04213181595006 4.695062793133971),6
7,188,LA REBAJA DROGUERIA NO 82 BOGOTA,Comercio al por mayor y al por menor; reparaci...,AK 15 NO. 100 20,4.685867,-74.047640,POINT (-74.04764047481679 4.685866650782341),7
8,25,CONSULTORIO DE FISIOTERAPIA III MILENIO,Actividades de atencionde la salud humana y de...,CLL 116 . 9-02,4.694661,-74.034799,POINT (-74.03479880989529 4.694661133263812),8
9,79,SUPERMERCADO JOSE G.M.,Comercio al por mayor y al por menor; reparaci...,CR 21 NO. 195A-17,4.773825,-74.041213,POINT (-74.04121290319479 4.773824757701787),9


In [91]:
mapa = folium.Map(location=[Centroides.head(1)['Latitude'].iloc[0],Centroides.head(1)['Longitude'].iloc[0]],zoom_start=13,tiles='cartodbpositron')



In [92]:
len(Centroides)

15

### Crear Mapa

In [93]:

mapa = folium.Map(location=[Centroides.head(1)['Latitude'].iloc[0],Centroides.head(1)['Longitude'].iloc[0]],zoom_start=13,tiles='cartodbpositron')


Vector_colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

##### ------- Graficar Centroides ------ ###### 
for j in range(0,len(Centroides)):
    punto = Centroides.iloc[j] 
    folium.Marker(
        [punto['Latitude'],punto['Longitude']],icon=folium.Icon(color=Vector_colores[j], icon='cloud'),tooltip=punto['Nombre']
    ).add_to(mapa)
    
    
    ##### ------- Graficar vecinos de los centroides ------ ###### 
    Vecinos_cluster = Lugares_usaquen_sample[Lugares_usaquen_sample['Grupo'] == punto['Grupo']]
    
    ##### ------- Excluir al centro ------ ####
    Vecinos_cluster = Vecinos_cluster[Vecinos_cluster['ID'] != punto['ID']] 
    for v in range(0,len(Vecinos_cluster)):
        punto_vecino = Vecinos_cluster.iloc[v] 
        folium.Marker(
            [punto_vecino['Latitude'],punto_vecino['Longitude']],icon=folium.Icon(color=Vector_colores[j], icon='book'),tooltip=punto_vecino['Nombre']
        ).add_to(mapa)





In [95]:
mapa